In [ ]:
# prerequisite: 
# run draw3aHists.ipynb, drawPadVetoedHists.ipynb and drawProtonHists.ipynb 
# with output "res_alpha_corrected.dat", "res_padVetoed_corrected.dat" and "res_proton_corrected.dat"
# more todo

In [ ]:
import os
exec(open(os.path.expanduser('~') + "/matplotlibpreamble.py").read())

In [ ]:
raw = np.array([])
for batch in uproot.iterate(("../data/unpacked/098_*.root:h101",
                             "../data/unpacked/099_*.root:h101",
                             "../data/unpacked/100_*.root:h101",
                             "../data/unpacked/101_*.root:h101"), expressions=["U2F_E"], cut="U2FI == 8"):
    raw = np.append(raw, ak.flatten(batch.U2F_E))

In [ ]:
base = "U2F8"
with uproot.open("../data/cal/3aha.root") as file:
    # data
    y1a, x1a = file[base].to_numpy()
    x0a = np.min(x1a)
    x1a = x1a[:-1] + 0.5
    # tspectrum
    y2a, x2a = file[base + "A"].to_numpy()
    x2a = x2a[:-1] + 0.5 + x0a
    # peak positions, in found order
    x3a, y3a = file[base + "P"].values()
    x3a = x3a + x0a
with uproot.open("../data/cal/pvha.root") as file:
    # data
    y1p, x1p = file[base].to_numpy()
    x0p = np.min(x1p)
    x1p = x1p[:-1] + 0.5
    # tspectrum
    y2p, x2p = file[base + "A"].to_numpy()
    x2p = x2p[:-1] + 0.5 + x0p
    # peak positions, in found order
    x3p, y3p = file[base + "P"].values()
    x3p = x3p + x0p

In [ ]:
xl1 = 0
xu1 = 900
xl2 = 1570
xu2 = 1900
fig, axes = prepare_two_column_figure(height_scale=1.6, fig_scale=1.5, nrows=3, ncols=2,
                                      gridspec_kw={'wspace': 0.05, 'width_ratios': [1., (xu2 - xl2)/(xu1 - xl1)],
                                                   'hspace': 0.08, 'height_ratios': [3, 2, 1]})

d = 2.5  # proportion of vertical to horizontal extent of the slanted line
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)



plt.sca(axes[0,0])
xpl = np.min(x1p)
xpu = np.max(x1p)
plt.hist(raw, bins=np.arange(xpl, xpu, (xpu - xpl)/len(x1p)), histtype='stepfilled', color='grey', edgecolor='k',
         label='Protons w/o veto on backing detector')

plt.fill_between(x1p, 0, y1p, edgecolor='k', facecolor='w', label='Protons from $^{25}\mathrm{Si}$')
plt.fill_between([-100, -99], 0, -100, edgecolor='k', facecolor='w', linestyle='--', label='Alphas from 3α source $\\times\,\\frac{1}{13}$')
plt.xticks(np.arange(0, 800+200, 200))
plt.xticks(np.arange(0, 800+100, 100), minor=True)
plt.yticks(np.arange(50, 150+50, 50))
plt.yticks(np.arange(10, 160+10, 10), minor=True)
plt.xlim(xl1, xu1)
plt.ylim(0, 160)

y3p = [109, 92, 64, 56, 45]
plt.plot(x3p[0:5], y3p, c=wr, ls='', marker='v', label='Peaks used for calibrations')
subs = ["1", "3", "2", "4", "5"]
for x, y, sub in zip(x3p[0:5], y3p, subs):
    plt.text(x, y+2, "$\mathrm{p_{%s}}$" % sub, va='bottom', ha='center', c=wr, size='small')

plt.gca().spines.right.set_visible(False)
plt.gca().yaxis.set_tick_params(right=False, which='both')
plt.gca().tick_params(labelbottom=False, labeltop=True)
plt.ylabel("Counts / channel")
plt.plot([1, 1], [0, 1], transform=plt.gca().transAxes, **kwargs)
handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 2, 3, 0]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
           fontsize='x-small')



plt.sca(axes[0,1])
plt.fill_between(x1a[x1a > xl2 + 5], 0, y1a[x1a > xl2 + 5]/13, edgecolor='k', facecolor='w', linestyle='--')
plt.xticks(np.arange(1800, 1800+200, 200))
plt.xticks(np.arange(1600, 1900+100, 100), minor=True)
plt.yticks(np.arange(50, 150+50, 50))
plt.yticks(np.arange(10, 160+10, 10), minor=True)
plt.xlim(xl2, xu2)
plt.ylim(0, 160)

y3a = [106, 80, 20]
plt.plot(x3a, y3a, c=wr, ls='', marker='v')
subs = ["1", "2", "3"]
for x, y, sub in zip(x3a, y3a, subs):
    plt.text(x, y+2, "$\mathrm{\\alpha_{%s}}$" % sub, va='bottom', ha='center', c=wr, size='small')

plt.gca().spines.left.set_visible(False)
plt.gca().yaxis.set_tick_params(left=False, which='both')
plt.gca().tick_params(labelleft=False)
plt.gca().tick_params(labelbottom=False, labeltop=True)
plt.plot([0, 0], [0, 1], transform=plt.gca().transAxes, **kwargs)



XP = np.linspace(xl1, xu1, 100)
XA = np.linspace(xl2, xu2, 100)
a1 = 3.19597731
b1 = -44.35599225
a2 = 3.18919124
b2 = -70.92673806
a1err = 1.01716708e-05
b1err = 2.81267138e+00
ab1err = -4.82953329e-03
a2err = 2.13926055e-05
b2err = 6.46689773e+01
ab2err = -3.71518970e-02
def lin(x, a, b):
    return a*x + b
def lin_err(x, aerr, berr, aberr):
    return np.sqrt(aerr*x**2* + berr + x*aberr)

plt.sca(axes[1,0])
plt.plot(x3p[0:5], lin(x3p[0:5], a1, b1)/1e3, 'ko', label='Protons from $^{25}\mathrm{Si}$')
plt.plot(XP, lin(XP, a1, b1)/1e3, 'k-', lw=0.6, label='Fit to protons')
plt.plot(-50, -50, 'ko', markerfacecolor='w', label='Alphas from 3α source')
plt.plot(XP, lin(XP, a2, b2)/1e3, 'k--', lw=0.6, label='Fit to alphas')
plt.xlim(xl1, xu1)
plt.ylim(-0.5, 6.5)

axin = plt.gca().inset_axes([0.1, 0.4, 0.30, 0.30])
f1 = (0 < XP) & (XP < 50)
axin.plot(XP[f1], lin(XP[f1], a1, b1)/1e3, 'k-')
axin.plot(XP[f1], lin(XP[f1], a2, b2)/1e3, 'k--')
axin.set_xticks([])
axin.set_yticks([])
plt.gca().indicate_inset_zoom(axin, edgecolor="black")
axin.set_title("27 keV difference", fontsize='x-small')
plt.yticks(np.arange(0, 6, 1), minor=True)
plt.xticks(np.arange(0, 800+200, 200))
plt.xticks(np.arange(0, 800+100, 100), minor=True)
plt.legend(fontsize='x-small')

plt.gca().spines.right.set_visible(False)
plt.gca().yaxis.set_tick_params(right=False, which='both')
plt.gca().tick_params(labelbottom=False, labeltop=False)
plt.ylabel("Energy (MeV)")
plt.plot([1, 1], [0, 1], transform=plt.gca().transAxes, **kwargs)



plt.sca(axes[1,1])

plt.plot(XA, lin(XA, a1, b1)/1e3, 'k-', lw=0.6)
plt.plot(XA, lin(XA, a2, b2)/1e3, 'k--', lw=0.6)
plt.plot(x3a[0:3], lin(x3a[0:3], a2, b2)/1e3, 'ko', markerfacecolor='w', zorder=10)
plt.xlim(xl2, xu2)
plt.ylim(-0.5, 6.5)

axin = plt.gca().inset_axes([0.1, 0.2, 0.30/((xu2 - xl2)/(xu1 - xl1)), 0.30])
f2 = (1850 < XA) & (XA < 1900)
axin.plot(XA[f2], lin(XA[f2], a1, b1)/1e3, 'k-')
axin.plot(XA[f2], lin(XA[f2], a2, b2)/1e3, 'k--')
axin.set_xticks([])
axin.set_yticks([])
plt.gca().indicate_inset_zoom(axin, edgecolor="black")
axin.set_title("39 keV difference", y=-0.34, fontsize='x-small')
plt.yticks(np.arange(0, 6, 1), minor=True)
plt.xticks(np.arange(1800, 1800+200, 200))
plt.xticks(np.arange(1600, 1900+100, 100), minor=True)

plt.gca().spines.left.set_visible(False)
plt.gca().yaxis.set_tick_params(left=False, which='both')
plt.gca().tick_params(labelleft=False)
plt.gca().tick_params(labelbottom=False, labeltop=False)
plt.plot([0, 0], [0, 1], transform=plt.gca().transAxes, **kwargs)



plt.sca(axes[2,0])
plt.plot([xl1, xu1 - 20], [0, 0], ls='--', c='grey', lw=0.5)
plt.plot(x3p[0:5], [1.57276064, -2.10437255, -0.41702563, 0.1081656, 0.84047203], 'ko')
plt.xlim(xl1, xu1)
plt.ylim(-3.5, 3.5)
plt.yticks(np.arange(-2, 2+2, 2))
plt.yticks(np.arange(-3, 3+1, 1), minor=True)
plt.xticks(np.arange(0, 800+200, 200))
plt.xticks(np.arange(0, 800+100, 100), minor=True)

plt.gca().spines.right.set_visible(False)
plt.gca().yaxis.set_tick_params(right=False, which='both')
plt.ylabel("Residuals (keV)")
plt.plot([1, 1], [0, 1], transform=plt.gca().transAxes, **kwargs)



plt.sca(axes[2,1])
plt.plot([20 + xl2, xu2], [0, 0], ls='--', c='grey', lw=0.5)
plt.plot(x3a[0:3], [0.26701039, -0.54474203, 0.27773164], 'ko', markerfacecolor='w')
plt.xlim(xl2, xu2)
plt.ylim(-3.5, 3.5)
plt.yticks(np.arange(-2, 2+2, 2))
plt.yticks(np.arange(-3, 3+1, 1), minor=True)
plt.xticks(np.arange(1800, 1800+200, 200))
plt.xticks(np.arange(1600, 1900+100, 100), minor=True)

plt.gca().spines.left.set_visible(False)
plt.gca().yaxis.set_tick_params(left=False, which='both')
plt.gca().tick_params(labelleft=False)
plt.plot([0, 0], [0, 1], transform=plt.gca().transAxes, **kwargs)

fig.text(0.5, 0.06, 'Channel number', rotation='horizontal', ha='center', va='center')
plt.savefig("cal-comp-3a-pv.pdf", bbox_inches='tight')
plt.show()

In [ ]:
print(lin(0, a1, b1) - lin(0, a2, b2))
print(lin(1900, a1, b1) - lin(1900, a2, b2))

In [ ]:
E0s = np.array([385.8, 905.4, 1847, 2077, 2218])
Es = np.loadtxt('correctedEnergiesU1U2U3U6.dat').flatten()

E0sa = np.array([5155.4, 5485.74, 5804.96])
Esa = np.array([])
Esa = np.append(Esa, np.loadtxt('corrected3aEnergiesU1U2.dat').flatten())
Esa = np.append(Esa, np.loadtxt('corrected3aEnergiesU3U4.dat').flatten())
Esa = np.append(Esa, np.loadtxt('corrected3aEnergiesU5.dat').flatten())
Esa = np.append(Esa, np.loadtxt('corrected3aEnergiesU6.dat').flatten())

In [ ]:
fig, axes = prepare_two_column_figure(height_scale=0.5, fig_scale=1.5, nrows=1, ncols=2, sharey=True, 
                                    gridspec_kw={'wspace': 0.05})

N = int(len(Es)/5)
print(N)
M = int(len(Esa)/3)
print(M)

plt.sca(axes[0])
plt.plot(E0s[0]*np.ones(N)/1e3, E0s[0]*np.ones(N) - Es[0::5], 'k.')
plt.plot(E0s[1]*np.ones(N)/1e3, E0s[1]*np.ones(N) - Es[1::5], 'k.')
plt.plot(E0s[2]*np.ones(N)/1e3, E0s[2]*np.ones(N) - Es[2::5], 'k.')
plt.plot(E0s[3]*np.ones(N)/1e3, E0s[3]*np.ones(N) - Es[3::5], 'k.')
plt.plot(E0s[4]*np.ones(N)/1e3, E0s[4]*np.ones(N) - Es[4::5], 'k.')
plt.text(0.05+E0s[0]/1e3, np.max(E0s[0]*np.ones(N) - Es[0::5]), "$\mathrm{p}_1$", ha="left", va="top")
plt.text(0.05+E0s[1]/1e3, np.max(E0s[1]*np.ones(N) - Es[1::5]), "$\mathrm{p}_2$", ha="left", va="top")
plt.text(E0s[2]/1e3-0.05, np.max(E0s[2]*np.ones(N) - Es[2::5]), "$\mathrm{p}_3$", ha="right", va="top")
plt.text(E0s[3]/1e3-0.03, np.max(E0s[3]*np.ones(N) - Es[3::5]), "$\mathrm{p}_4$", ha="right", va="top")
plt.text(E0s[4]/1e3, np.max(E0s[4]*np.ones(N) - Es[4::5])+1, "$\mathrm{p}_5$", ha="center", va="bottom")
plt.xlabel("$E_{\mathrm{p}}$ (MeV)")
plt.ylabel("$E_{\mathrm{p}} - E_{\mathrm{dep}}$ (keV)")
plt.xlim(0.300, 2.300)
plt.plot([.350, 2.300], [0, 0], ls='--', c='grey', lw=0.5)
plt.ylim(-5, 90)

plt.sca(axes[1])
plt.plot(E0sa[0]*np.ones(M)/1e3, E0sa[0]*np.ones(M) - Esa[0::3], 'k.')
plt.text(0.02+E0sa[0]/1e3, 2+np.max(E0sa[0]*np.ones(M) - Esa[0::3]), "$\mathrm{\\alpha}_1$", ha="left", va="top")
plt.plot(E0sa[1]*np.ones(M)/1e3, E0sa[1]*np.ones(M) - Esa[1::3], 'k.')
plt.text(0.02+E0sa[1]/1e3, 2+np.max(E0sa[1]*np.ones(M) - Esa[1::3]), "$\mathrm{\\alpha}_2$", ha="left", va="top")
plt.plot(E0sa[2]*np.ones(M)/1e3, E0sa[2]*np.ones(M) - Esa[2::3], 'k.')
plt.text(E0sa[2]/1e3-0.02, 2+np.max(E0sa[2]*np.ones(M) - Esa[2::3]), "$\mathrm{\\alpha}_3$", ha="right", va="top")
plt.xlabel("$E_{\mathrm{\\alpha}}$ (MeV)")
plt.ylabel("$E_{\mathrm{\\alpha}} - E_{\mathrm{dep}}$ (keV)")
plt.gca().tick_params(labelleft=False, labelright=True)
plt.gca().yaxis.set_label_position("right")
plt.xlim(5.05, 5.9)
plt.plot([5.05, 5.9], [0, 0], ls='--', c='grey', lw=0.5)

plt.savefig("cal-eloss.pdf", bbox_inches="tight")

plt.show()

In [ ]:
resp0 = np.loadtxt("res_padVetoed_uncorrected.dat")
resp = np.loadtxt("res_padVetoed_corrected.dat")
resa = np.loadtxt("res_alpha_corrected.dat")
resP0 = np.loadtxt("res_proton_uncorrected.dat")
resP = np.loadtxt("res_proton_corrected.dat")

In [ ]:
print(len(resp0)/5, len(resp)/5, len(resa)/3, len(resP0)/4, len(resP)/4)

In [ ]:
print(np.min(resp0), np.max(resp0))
print(np.min(resp), np.max(resp))
print(np.min(resa), np.max(resa))
print(np.min(resP0), np.max(resP0))
print(np.min(resP), np.max(resP))

In [ ]:
fig, axes = prepare_two_column_figure(height_scale=0.5, fig_scale=1.5, nrows=1, ncols=2, sharey=True, 
                                    gridspec_kw={'wspace': 0.05})

bp = np.arange(-10.5, 10.5+0.5, 0.8)
ba = np.arange(-4, 4+0.1, 0.2)

plt.sca(axes[0])
plt.hist(resp, bins=bp, histtype='step', color='k', ls='-', lw=1.2, label='Protons w/ \nenergy corrections')
plt.hist(resp0, bins=bp, histtype='step', color=wr, ls='--', lw=2, label='Protons w/o \nenergy corrections')
# plt.ylim(0.4, 120)
plt.xlabel("Residuals (keV)")
plt.ylabel("Counts / 0.8 keV")
plt.legend(fontsize='x-small', loc='upper right', bbox_to_anchor=(1.2, 1))
axes[0].set_zorder(100)

plt.sca(axes[1])
plt.hist(np.array([resa[0::3], resa[2::3]]).flatten(), bins=ba, histtype='step', color=wdb, ls='-', lw=1.2, label='$\mathrm{\\alpha}_1$, $\mathrm{\\alpha}_3$')
plt.hist(resa[1::3], bins=ba, histtype='step', color=wg, ls='--', lw=2, label='$\mathrm{\\alpha}_2$')
plt.xlabel("Residuals (keV)")
plt.ylabel("Counts / 0.2 keV")
plt.gca().tick_params(labelleft=False, labelright=True)
plt.gca().yaxis.set_label_position("right")
# plt.ylim(0.4, 120)
plt.legend(fontsize='x-small')

plt.savefig("res-distributions.pdf", bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = prepare_two_column_figure(height_scale=0.5, fig_scale=1.5, nrows=1, ncols=2, sharey=True, 
                                    gridspec_kw={'wspace': 0.05})

bp = np.arange(-14.5, 14.5+0.5, 0.8)
ba = np.arange(-4, 4+0.1, 0.2)

plt.sca(axes[0])
plt.hist(resP, bins=bp, histtype='step', color='k', ls='-', lw=1.2, label='Protons w/ \nenergy corrections')
plt.hist(resP0, bins=bp, histtype='step', color=wr, ls='--', lw=2, label='Protons w/o \nenergy corrections')
# plt.ylim(0.4, 120)
plt.xlabel("Residuals (keV)")
plt.ylabel("Counts / 0.8 keV")
plt.legend(fontsize='x-small', loc='upper right', bbox_to_anchor=(1.2, 1))
axes[0].set_zorder(100)

plt.sca(axes[1])
plt.hist(np.array([resa[0::3], resa[2::3]]).flatten(), bins=ba, histtype='step', color=wdb, ls='-', lw=1.2, label='$\mathrm{\\alpha}_1$, $\mathrm{\\alpha}_3$')
plt.hist(resa[1::3], bins=ba, histtype='step', color=wg, ls='--', lw=2, label='$\mathrm{\\alpha}_2$')
plt.xlabel("Residuals (keV)")
plt.ylabel("Counts / 0.2 keV")
plt.gca().tick_params(labelleft=False, labelright=True)
plt.gca().yaxis.set_label_position("right")
# plt.ylim(0.4, 120)
plt.legend(fontsize='x-small')

#plt.savefig("res-distributions.pdf", bbox_inches='tight')
plt.show()

In [ ]:
def corr(x, y):
    mx = np.mean(x)
    my = np.mean(y)
    sx = np.std(x, ddof=1)
    sy = np.std(y, ddof=1)
    return np.sum(np.dot(x - mx, y - my))/(len(x) - 1)/sx/sy

In [ ]:
print(corr(resp[1::5], resp[0::5]))

In [ ]:
fig, axes = prepare_two_column_figure(height_scale=1.3, fig_scale=1.5, nrows=4, ncols=4, 
                                    #sharex=True, sharey=True,
                                    gridspec_kw={'wspace': 0.05, 'hspace': 0.05})

plt.sca(axes[0, 0])
plt.plot(resp[1::5], resp[0::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_1$ vs. $\mathrm{p}_2$\n$\\rho$ = %1.2f' 
         % corr(resp[1::5], resp[0::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelbottom=False)
plt.sca(axes[1, 0])
plt.plot(resp[2::5], resp[0::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_1$ vs. $\mathrm{p}_3$\n$\\rho$ = %1.2f' 
         % corr(resp[2::5], resp[0::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelbottom=False)
plt.sca(axes[2, 0])
plt.plot(resp[3::5], resp[0::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_1$ vs. $\mathrm{p}_4$\n$\\rho$ = %1.2f' 
         % corr(resp[3::5], resp[0::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelbottom=False)
plt.sca(axes[3, 0])
plt.plot(resp[4::5], resp[0::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_1$ vs. $\mathrm{p}_5$\n$\\rho$ = %1.2f' 
         % corr(resp[4::5], resp[0::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)

plt.sca(axes[1, 1])
plt.plot(resp[2::5], resp[1::5], 'k.')
plt.text(0.95, 0.05, '$\mathrm{p}_2$ vs. $\mathrm{p}_3$\n$\\rho$ = %1.2f' 
         % corr(resp[2::5], resp[1::5]),
         horizontalalignment='right', verticalalignment='bottom', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelbottom=False, labelleft=False)
plt.sca(axes[2, 1])
plt.plot(resp[3::5], resp[1::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_2$ vs. $\mathrm{p}_4$\n$\\rho$ = %1.2f' 
         % corr(resp[3::5], resp[1::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelbottom=False, labelleft=False)
plt.sca(axes[3, 1])
plt.plot(resp[4::5], resp[1::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_2$ vs. $\mathrm{p}_5$\n$\\rho$ = %1.2f' 
         % corr(resp[4::5], resp[1::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelleft=False)

plt.sca(axes[2, 2])
plt.plot(resp[3::5], resp[2::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_3$ vs. $\mathrm{p}_4$\n$\\rho$ = %1.2f' 
         % corr(resp[3::5], resp[2::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelbottom=False, labelleft=False)
plt.sca(axes[3, 2])
plt.plot(resp[4::5], resp[2::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_3$ vs. $\mathrm{p}_5$\n$\\rho$ = %1.2f' 
         % corr(resp[4::5], resp[2::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelleft=False)

plt.sca(axes[3, 3])
plt.plot(resp[4::5], resp[3::5], 'k.')
plt.text(0.95, 0.95, '$\mathrm{p}_4$ vs. $\mathrm{p}_5$\n$\\rho$ = %1.2f' 
         % corr(resp[4::5], resp[3::5]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-11, 11)
plt.ylim(-11, 11)
plt.xticks(np.arange(-8, 8+8, 8))
plt.xticks(np.arange(-10, 10+2, 2), minor=True)
plt.yticks(np.arange(-8, 8+8, 8))
plt.yticks(np.arange(-10, 10+2, 2), minor=True)
plt.gca().tick_params(labelleft=False)

plt.sca(axes[0, 3])
plt.plot(resa[1::3], resa[0::3], 'k.')
plt.text(0.95, 0.95, '$\mathrm{\\alpha}_1$ vs. $\mathrm{\\alpha}_2$\n$\\rho$ = %1.2f' 
         % corr(resa[1::3], resa[0::3]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.xticks(np.arange(-4, 4+2, 4))
plt.xticks(np.arange(-4, 4+1, 1), minor=True)
plt.yticks(np.arange(-4, 4+2, 4))
plt.yticks(np.arange(-4, 4+1, 1), minor=True)
plt.gca().tick_params(labelbottom=False, labelleft=False, labeltop=True, labelright=True)
plt.sca(axes[1, 3])
plt.plot(resa[2::3], resa[0::3], 'k.')
plt.text(0.95, 0.95, '$\mathrm{\\alpha}_1$ vs. $\mathrm{\\alpha}_3$\n$\\rho$ = %1.2f' 
         % corr(resa[2::3], resa[0::3]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.xticks(np.arange(-4, 4+2, 4))
plt.xticks(np.arange(-4, 4+1, 1), minor=True)
plt.yticks(np.arange(-4, 4+2, 4))
plt.yticks(np.arange(-4, 4+1, 1), minor=True)
plt.gca().tick_params(labelbottom=False, labelleft=False, labelright=True)

plt.sca(axes[0, 2])
plt.plot(resa[2::3], resa[1::3], 'k.')
plt.text(0.95, 0.95, '$\mathrm{\\alpha}_2$ vs. $\mathrm{\\alpha}_3$\n$\\rho$ = %1.2f' 
         % corr(resa[2::3], resa[1::3]),
         horizontalalignment='right', verticalalignment='top', 
         transform=plt.gca().transAxes, fontsize='x-small')
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.xticks(np.arange(-4, 4+2, 4))
plt.xticks(np.arange(-4, 4+1, 1), minor=True)
plt.yticks(np.arange(-4, 4+2, 4))
plt.yticks(np.arange(-4, 4+1, 1), minor=True)
plt.gca().tick_params(labelbottom=False, labelleft=False, labeltop=True)

axes[0, 1].remove()
axes[1, 2].remove()
axes[2, 3].remove()

fig.text(0.06, 0.5, 'Residuals (keV)', rotation='vertical', ha='center', va='center')
fig.text(0.5, 0.06, 'Residuals (keV)', rotation='horizontal', ha='center', va='center')

plt.savefig("res-comp-3a-pv.pdf", bbox_inches='tight')
print(len(resp[0::5]))
print(len(resa[0::3]))

In [ ]:
X = np.linspace(0, 3000, 200)
plt.plot(X, lin(X, a1, b1) - lin(X, a2, b2))

In [ ]:
Ep = Ea = np.array([])
for batch in uproot.iterate("../data/deltaEcontained/21mg.root:a", expressions=["E"], 
                            cut="((id == 0) | (id == 1) | (id == 2)) & (1 < FI) & (FI < 16) & (1 < BI) & (BI < 16)"):
    Ep = np.append(Ep, ak.flatten(batch.E))
for batch in uproot.iterate("../data/deltaEcontainedAlpha/21mg.root:a", expressions=["E"], 
                            cut="((id == 0) | (id == 1) | (id == 2)) & (1 < FI) & (FI < 16) & (1 < BI) & (BI < 16)"):
    Ea = np.append(Ea, ak.flatten(batch.E))

In [ ]:
fig = prepare_one_column_figure(height_scale=1.0, fig_scale=2.0)
b = np.arange(0.5, 3.0, 0.004)
#plt.hist(Ep/1e3, bins=b, histtype='step', color='k', ls='-',  lw=1.2, label='Using $^{25}\mathrm{Si}$ proton calibration')
#plt.hist(Ea/1e3, bins=b, histtype='step', color=wb, ls='--', lw=1.0, label='Using triple-α source calibration')
plt.hist(Ep/1e3, bins=b, histtype='step', color='k', ls='-',  lw=1.2, label='Using $^{25}\mathrm{Si}$ p cal.')
plt.hist(Ea/1e3, bins=b, histtype='step', color=wb, ls='--', lw=1.0, label='Using triple-α cal.')
#plt.plot(-100, -100, color='k', ls='-', lw=0.9, label='$^{21}\mathrm{Mg}$ protons from literature')
plt.xlim(0.45, 3.0)
plt.xticks(np.arange(0.5, 3.0+0.1, 0.1), minor=True)
plt.xticks(np.arange(0.5, 3.0+0.5, 0.5))
#plt.yticks(np.arange(0, 4000, 500), minor=True)
plt.ylim(0.6, 2e5)
#plt.ylim(0, 3800)

xs = np.array([1.9383, 1.7730, 1.5019, 1.0590, 0.909, 1.243])
ys = np.array([4.3e3, 2.9e3, 3.7e2, 2.9e2, 3.5e2, 1.3e3])
subs = ["VI", "V", "IV", "II", "I", "III"]
subs_extra = ["", "", "", "", "*", "*"]
#plt.plot(xs, ys, c=wr, ls='', marker='v', label='$^{21}\mathrm{Mg}$ protons from literature')
plt.plot(xs, ys, c=wr, ls='', marker='v', label='$^{21}\mathrm{Mg}$ p from lit.')
for x, y, sub, sub_e in zip(xs, ys, subs, subs_extra):
    plt.text(x, 1.2*y, "$\mathrm{p_{%s}}%s$" % (sub, sub_e), va='bottom', ha='center', c=wr, size='x-small')
#plt.text(1.9383, 4e3)
#plt.text(1.7730, 2.5e3)
#plt.text(1.5019, 3.5e2)

plt.yscale('log')

#b2 = np.arange(1.65, 2.1, 0.004)
#axin = plt.gca().inset_axes([1.05, 0.4, 0.50, 0.30])
#axin.hist(Ep/1e3, bins=b2, histtype='step', color=wb, ls='-',  lw=1.2)
#axin.hist(Ea/1e3, bins=b2, histtype='step', color=wr, ls='--', lw=1.0)
#axin.vlines([1.9383, 1.7730], 0, [3800, 2500], color='k', ls='-', lw=0.9)
#axin.tick_params(labelleft=False, labelright=True, labelsize='xx-small')
#axin.set_xticks(np.arange(1.7, 2.1, 0.2))
#axin.set_xticks(np.arange(1.65, 2.1+0.05, 0.05), minor=True)
#axin.set_yticks(np.arange(0, 4000, 2000))
#axin.set_yticks(np.arange(0, 4000, 500), minor=True)
#axin.get_yaxis().set_visible(False)
#axin.get_xaxis().set_visible(False)

#fig.gca().indicate_inset_zoom(axin, edgecolor="black")
plt.xlabel("$E_{\mathrm{p}}$ (MeV)")
plt.ylabel("Counts / 4 keV")
#plt.legend(fontsize='xx-small', bbox_to_anchor=(1., 1.))
plt.legend(fontsize='xx-small', loc='upper left', ncol=2)
plt.savefig('padVetoedCalComp.pdf', bbox_inches='tight')
plt.show()

In [ ]:
from scipy.optimize import curve_fit
def gaus(x, mu, sigma, A):
    return A*np.exp(-0.5*(x-mu)**2/sigma**2)/sigma/np.sqrt(2*np.pi)

In [ ]:
X1 = np.linspace(1.73, 1.82, 100)
data1 = plt.hist(Ep/1e3, bins=b)
y1 = data1[0]
x1 = data1[1]
x1 = x1[0:-1] + (x1[1] - x1[0])/2
plt.xlim(1.73, 1.82)
f1 = (1.73 <= x1) & (x1 <= 1.82)
p1, pc1 = curve_fit(gaus, x1[f1], y1[f1], p0=(1.78, 5, 2000), sigma=np.sqrt(y1[f1]), absolute_sigma=True)
plt.plot(X1, gaus(X1, *p1))
plt.show()

X2 = np.linspace(1.7, 1.81, 100)
data2 = plt.hist(Ea/1e3, bins=b)
y2 = data2[0]
x2 = data2[1]
x2 = x2[0:-1] + (x2[1] - x2[0])/2
plt.xlim(1.7, 1.81)
f2 = (1.7 <= x2) & (x2 <= 1.81)
p2, pc2 = curve_fit(gaus, x2[f2], y2[f2], p0=(1.75, 5, 2000), sigma=np.sqrt(y2[f2]), absolute_sigma=True)
plt.plot(X2, gaus(X2, *p2))
plt.show()

print(p1[1], p2[1])
print(2*np.sqrt(2*np.log(2))*p1[1], 2*np.sqrt(2*np.log(2))*p2[1])
print("FWHM from 50 keV down to 42 keV on this particular peak")
print("relative improvement of", 100*(50 - 42)/50, "%")

In [ ]:
X3 = np.linspace(1.88, 1.99, 100)
plt.hist(Ep/1e3, bins=b)
plt.xlim(1.88, 1.99)
f3 = (1.88 <= x1) & (x1 <= 1.99)
p3, pc3 = curve_fit(gaus, x1[f3], y1[f3], p0=(1.93, 5, 2000), sigma=np.sqrt(y1[f3]), absolute_sigma=True)
plt.plot(X3, gaus(X3, *p3))
plt.show()

X4 = np.linspace(1.86, 1.97, 100)
plt.hist(Ea/1e3, bins=b)
plt.xlim(1.86, 1.97)
f4 = (1.86 <= x2) & (x2 <= 1.97)
p4, pc4 = curve_fit(gaus, x2[f4], y2[f4], p0=(1.90, 5, 2000), sigma=np.sqrt(y2[f4]), absolute_sigma=True)
plt.plot(X4, gaus(X4, *p4))
plt.show()

print(p3[1], p3[1])
print(2*np.sqrt(2*np.log(2))*p3[1], 2*np.sqrt(2*np.log(2))*p4[1])
print("FWHM from 60 keV down to 55 keV on this particular peak")
print("relative improvement of", 100*(60 - 55)/60, "%")

In [ ]:
ap = bp = aa = ba = np.array([])
prefixes = ["U1.cal", "U2.cal", "U3.cal", "U4.cal"]
# prefixes = ["U2.cal"]
for pref in prefixes:
    b, a = np.loadtxt("../setup/%s" % pref, unpack=True)
    ap = np.append(ap, a)
    bp = np.append(bp, b)
    b, a = np.loadtxt("../setup/%s.alpha" % pref, unpack=True)
    aa = np.append(aa, a)
    ba = np.append(ba, b)

In [ ]:
def lin(x, a, b):
    return a*x + b

In [ ]:
fig = prepare_two_column_figure(height_scale=0.6, fig_scale=2.0)

i = 1
for a0, b0, a1, b1 in zip(aa, ba, ap, bp):
    if a1 == 0 or a0 == 0:
        i +=1
        continue
    plt.plot([i, i], [lin(   0, a1, b1) - lin(   0, a0, b0),
                      lin(1900, a1, b1) - lin(1900, a0, b0)], 'k.-')
    plt.plot(i, lin(1900, a1, b1) - lin(1900, a0, b0), marker='s', color=wr, ms=4)
    i += 1

texts = ["U1p", "U1n", "U2p", "U2n", "U3p", "U3n", "U4p", "U4n"]
xt    = [9, 25, 41, 57, 73, 89, 105, 121]
yt    = [-50, -50, -50, 100, 100, 100, -50, 100]
for text, x, y in zip(texts, xt, yt):
    plt.text(x, y, text, ha='center', va='center')
plt.xlabel('Channel number')
plt.ylabel('$\Delta E_{\mathrm{dep}}(x)$ (keV)')
plt.xlim(0.1, 130)
plt.plot(-10, 0, 'k.', label='$x=0$')
plt.plot(-10, 0, marker='s', linestyle='', color=wr, ms=4, label='$x=1900$')
plt.text(37.7+0.3, 55, '→', rotation=-90.)
plt.text(40, 85, 'Strip of\nfigure 7.5', ha='center', fontsize='xx-small')
plt.text(101.7+0.3, 99, '→', rotation=-90.)
plt.text(103, 125, 'Strip of\nfigure 7.11', ha='left', fontsize='xx-small')
plt.legend()
plt.savefig('Edep-diff.pdf', bbox_inches='tight')
plt.show()

In [ ]:
base = "U4F7"
with uproot.open("../data/cal/3aha.root") as file:
    # data
    y1a_, x1a_ = file[base].to_numpy()
    x0a_ = np.min(x1a_)
    x1a_ = x1a_[:-1] + 0.5
    # tspectrum
    y2a_, x2a_ = file[base + "A"].to_numpy()
    x2a_ = x2a_[:-1] + 0.5 + x0a_
    # peak positions, in found order
    x3a_, y3a_ = file[base + "P"].values()
    x3a_ = x3a_ + x0a_
for i in range(len(x1a_)):
    xtmp = x1a_[i]
    if 1350 <= xtmp and xtmp <= 1380:
        y1a_[i] = 2 + np.random.uniform(-2, 2)
    if 1450 <= xtmp and xtmp <= 1475:
        y1a_[i] = 2 + np.random.uniform(-2, 2)
with uproot.open("../data/cal/pha.root") as file:
    # data
    y1p_, x1p_ = file[base].to_numpy()
    x0p_ = np.min(x1p_)
    x1p_ = x1p_[:-1] + 0.5
    # tspectrum
    y2p_, x2p_ = file[base + "A"].to_numpy()
    x2p_ = x2p_[:-1] + 0.5 + x0p_
    # peak positions, in found order
    x3p_, y3p_ = file[base + "P"].values()
    x3p_ = x3p_ + x0p_

In [ ]:
xl1_ = 300
xu1_ = 1800
xl2_ = 1350
xu2_ = 1650
fig, axes = prepare_two_column_figure(height_scale=1.6, fig_scale=1.5, nrows=3, ncols=1, sharex=True,
                                      gridspec_kw={'hspace': 0.08, 'height_ratios': [3, 2, 1]})

plt.sca(axes[0])
xpl = np.min(x1p_)
xpu = np.max(x1p_)

plt.fill_between(x1p_, 0, y1p_, edgecolor='k', facecolor='w', label='Protons from $^{21}\mathrm{Mg}$')

plt.yticks(np.arange(50, 100+50, 50))
plt.yticks(np.arange(10, 130+10, 10), minor=True)
# plt.xlim(xl1, xu1)
plt.ylim(0, 130)

# plt.gca().spines.right.set_visible(False)
# plt.gca().yaxis.set_tick_params(right=False, which='both')
plt.gca().tick_params(labelbottom=False, labeltop=True)
plt.ylabel("Counts / channel")
# plt.plot([1, 1], [0, 1], transform=plt.gca().transAxes, **kwargs)
# handles, labels = plt.gca().get_legend_handles_labels()
# order = [1, 2, 3, 0]
# plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
#            fontsize='x-small')

plt.fill_between(x1a_[x1a_ > xl2_ + 5], 0, y1a_[x1a_ > xl2_ + 5]/13, edgecolor='k', facecolor='w', linestyle='--',
                label='Alphas from 3α source $\\times\,\\frac{1}{13}$')

x4p_ = np.array([x3p_[np.argsort(x3p_)][-1], x3p_[np.argsort(x3p_)][-2],
                 x3p_[np.argsort(y3p_)][-1], x3p_[np.argsort(y3p_)][-2]])
y4p_ = [12, 40, 110, 78]
plt.plot(x4p_, y4p_, c=wr, ls='', marker='v', label='Peaks used for calibrations')
subs = ["IIX", "VII", "VI", "V"]
for x, y, sub in zip(x4p_, y4p_, subs):
    plt.text(x, y+2, "$\mathrm{p_{%s}}$" % sub, va='bottom', ha='center', c=wr, size='small')
y3a_ = [110, 88, 23]
plt.plot(x3a_[0:3], y3a_, c=wr, ls='', marker='v')
subs = ["1", "2", "3"]
for x, y, sub in zip(x3a_[0:3], y3a_, subs):
    plt.text(x, y+2, "$\mathrm{\\alpha_{%s}}$" % sub, va='bottom', ha='center', c=wr, size='small')


plt.legend(fontsize='x-small', loc='upper center')
# plt.xticks(np.arange(1800, 1800+200, 200))
# plt.xticks(np.arange(1600, 1900+100, 100), minor=True)
# plt.yticks(np.arange(50, 150+50, 50))
# plt.yticks(np.arange(10, 160+10, 10), minor=True)
# plt.xlim(xl2, xu2)
# plt.ylim(0, 160)

# plt.gca().spines.left.set_visible(False)
# plt.gca().yaxis.set_tick_params(left=False, which='both')
# plt.gca().tick_params(labelleft=False)
# plt.gca().tick_params(labelbottom=False, labeltop=True)
# plt.plot([0, 0], [0, 1], transform=plt.gca().transAxes, **kwargs)



XP_ = np.linspace(xl1_, xu1_, 100)
a1_ = 3.66342003
b1_ = -39.6704681
a2_ = 3.63430057
b2_ = -61.61756198
a1err_ = 2.92422268e-06
b1err_ = 3.73928711e+00
ab1err_ = -2.94543330e-03
a2err_ = 5.06661114e-04
b2err_ = 1.17534795e+03
ab2err_ = -7.70799503e-01
def lin(x, a, b):
    return a*x + b
def lin_err(x, aerr, berr, aberr):
    return np.sqrt(aerr*x**2* + berr + x*aberr)

plt.sca(axes[1])
plt.plot(x4p_, lin(x4p_, a1_, b1_)/1e3, 'ko', label='Protons from $^{21}\mathrm{Mg}$')
plt.plot(XP_, lin(XP_, a1_, b1_)/1e3, 'k-', lw=0.6, label='Fit to protons')
plt.plot(XP_, lin(XP_, a2_, b2_)/1e3, 'k--', lw=0.6, label='Fit to alphas')
plt.plot(x3a_[0:3], lin(x3a_[0:3], a2_, b2_)/1e3, 'ko', markerfacecolor='w', zorder=10, 
         label='Alphas from 3α source')

axin = plt.gca().inset_axes([0.02, 0.5, 0.20, 0.30])
f1_ = (300 < XP_) & (XP_ < 350)
axin.plot(XP_[f1_], lin(XP_[f1_], a1_, b1_)/1e3, 'k-')
axin.plot(XP_[f1_], lin(XP_[f1_], a2_, b2_)/1e3, 'k--')
axin.set_xticks([])
axin.set_yticks([])
plt.gca().indicate_inset_zoom(axin, edgecolor="black")
axin.set_title("32 keV difference", fontsize='x-small')

axin = plt.gca().inset_axes([0.98-0.20, 0.2, 0.20, 0.30])
f2_ = (1750 < XP_) & (XP_ < 1800)
axin.plot(XP_[f2_], lin(XP_[f2_], a1_, b1_)/1e3, 'k-')
axin.plot(XP_[f2_], lin(XP_[f2_], a2_, b2_)/1e3, 'k--')
axin.set_xticks([])
axin.set_yticks([])
plt.gca().indicate_inset_zoom(axin, edgecolor="black")
axin.set_title("74 keV difference", y=-0.34, fontsize='x-small')

plt.yticks(np.arange(1, 6, 1), minor=True)
plt.legend(fontsize='x-small', loc='upper center', bbox_to_anchor=(0.45, 1))
plt.ylabel("Energy (MeV)")


plt.sca(axes[2])
plt.plot([xl1_, xu1_], [0, 0], ls='--', c='grey', lw=0.5)
plt.plot(x4p_, [1.39636508, -1.77668421, 0.89767393, -0.51735481], 'ko')
plt.plot(x3a_[0:3], [1.13063636, -2.32624076, 1.1956044], 'ko', markerfacecolor='w')
plt.ylabel("Residuals (keV)")
plt.ylim(-3.5, 3.5)
plt.yticks(np.arange(-2, 2+2, 2))
plt.yticks(np.arange(-3, 3+1, 1), minor=True)

plt.xlim(xl1_, xu1_)
plt.xticks(np.arange(400, 1800, 200))
plt.xticks(np.arange(300, 1800, 100), minor=True)

fig.text(0.5, 0.06, 'Channel number', rotation='horizontal', ha='center', va='center')
plt.savefig("cal-comp-3a-p.pdf", bbox_inches='tight')
plt.show()

In [ ]:
print(lin(350, a1_, b1_) - lin(350, a2_, b2_))
print(lin(1800, a1_, b1_) - lin(1800, a2_, b2_))